# review data to dataframe

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np

# 1. google crawrling
## 1.1 googl main

In [604]:
#https://chromedriver.storage.googleapis.com/index.html?path=80.0.3987.106/
driver = webdriver.Chrome('chromedriver.exe')

driver.get("http://www.google.com")
#assert "Google" in driver.title

element = driver.find_element_by_name('q')
element.clear()
element.send_keys('google map')
element.submit()

#### 브라우저 닫기

In [420]:
#driver.close()

## 1.2. google map site 들어가기

In [605]:
element = driver.find_element_by_xpath('//h3[@class="LC20lb DKV0Md"]')
element.click()

## 1.3. 원하는 장소 클릭
#### 검색어 입력

In [606]:
element = driver.find_element_by_xpath('//input[@autofocus="autofocus"]')
element.clear()

In [607]:
element.send_keys('타임스퀘어')
element.submit()


#### bs4로 html parser

In [608]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
all_address =soup.find_all('div', attrs = {'class': "suggest-left-cell"} )

real_address = '영등포동4가'
element_index=0
for i,address in enumerate(all_address):
#     print(address.contents[0], type(address.contents[0]))
    if real_address in address.text:
        element_index=i+1 
        break
        
print(element_index)

3


#### 주소 맞는 곳 클릭

In [609]:
# element = driver.find_element_by_xpath("//ul[@class='sbsb_b']/li[2]")
element = driver.find_element_by_xpath(f"//ul[@class='sbsb_b']/li[{element_index}]")
element.click()

## 1.4. 주소 가져오기

In [610]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

In [611]:
address =soup.find_all('span',attrs = {'class': "widget-pane-link"} )
address[2].text

'서울특별시 영등포구 영등포동 영중로 15'

## 1.5. review 가져오기

In [612]:
element = driver.find_element_by_xpath('//button[@class="jqnFjrOWMVU__button gm2-caption"]')
#element = driver.find_element_by_xpath('//span[@class="allxGeDnJMl__text gm2-button-alt"]')

element.click()

#### 스크롤 다운
- https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

In [616]:
element = driver.find_element_by_xpath('//span[@class="section-review-text"]')
element.click()

In [617]:
SCROLL_PAUSE_TIME = 2

In [618]:
while  True:
    html = driver.find_element_by_tag_name('html')
    text1 = html.text
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)
    
    html = driver.find_element_by_tag_name('html')
    text2 = html.text

    if text1 == text2:
        break
    else:
        continue

In [171]:
html = driver.page_source

#### bs4로 html parser

In [172]:
soup = BeautifulSoup(html, 'html.parser')

In [173]:
total_score =soup.find_all('div',attrs = {'class': "gm2-display-2"} )
total_score



[<div class="gm2-display-2" jsan="7.gm2-display-2" jstcache="964" style='font-family: "Google Sans Display", "Google Sans", Roboto, "Noto Sans KR", Arial, sans-serif;'>3.7</div>]

In [174]:
total_score[0].string

'3.7'

#### 개별 리뷰

In [175]:
total_review =soup.find_all('span',attrs = {'class': "section-review-text"} )
print(len(total_review))

2


In [176]:
review=[]
for i, review in enumerate(total_review):
    print(i, review.text)
    review.append(review.text)
print(len(review))

0 쾌적한 사무공간
1 깔끔하고 주차편하고....
2


#### 개별 별점

In [177]:
scores =soup.find_all('span',attrs = {'class': "section-review-stars", 'role': 'img'} )


In [178]:
score_list = []
for score in scores:
    print(score.attrs['aria-label'])
    score_list.append(score.attrs['aria-label'])
    
print(len(score_list))

 별표 4개 
 별표 4개 
2


> x-path 조절하기
- http://blog.naver.com/PostView.nhn?blogId=kiddwannabe&logNo=221323738780&categoryNo=35&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

> 스크롤 조절
https://m.blog.naver.com/kiddwannabe/221310063239